<a href="https://colab.research.google.com/github/RenataTNT/geekbrains_tasks/blob/master/Renata_irnazarova_InsuranceCase_Lesson_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Построить обобщенную линейную модель (GLM) для прогнозирования наступления страховых случаев на рассмотренных в ноутбуке данных. Подобрать необходимое распределение и тип связи, при необходимости ознакомиться с документацией H20. Придумать и использовать дополнительные факторы при построении модели (например, пересечения признаков или функции от них и т.д.). Оценить результаты построенной модели при помощи различных метрик (можно использовать и другие метрики помимо представленных в ноутбуке), проанализировать вероятные проблемы. Предложить способы их решения и/или попробовать их решить, улучшив результат.

## Предобработка данных

In [0]:
import numpy as np
import pandas as pd

In [6]:
# Загрузим набор данных

df = pd.read_csv('freMPL-R.csv', low_memory=False)
df = df.loc[df.Dataset.isin([5, 6, 7, 8, 9])]
df.drop('Dataset', axis=1, inplace=True)
df.dropna(axis=1, how='all', inplace=True)
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31777 entries, 0 to 31776
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Exposure           31777 non-null  float64
 1   LicAge             31777 non-null  int64  
 2   RecordBeg          31777 non-null  object 
 3   RecordEnd          17173 non-null  object 
 4   Gender             31777 non-null  object 
 5   MariStat           31777 non-null  object 
 6   SocioCateg         31777 non-null  object 
 7   VehUsage           31777 non-null  object 
 8   DrivAge            31777 non-null  float64
 9   HasKmLimit         31777 non-null  float64
 10  BonusMalus         31777 non-null  float64
 11  ClaimAmount        31777 non-null  float64
 12  ClaimInd           31777 non-null  float64
 13  ClaimNbResp        31777 non-null  float64
 14  ClaimNbNonResp     31777 non-null  float64
 15  ClaimNbParking     31777 non-null  float64
 16  ClaimNbFireTheft   317

In [7]:
df.head()

,Exposure,LicAge,RecordBeg,RecordEnd,Gender,MariStat,SocioCateg,VehUsage,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea
0,0.083,332,2004-01-01,2004-02-01,Male,Other,CSP50,Professional,46.0,0.0,50.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0
1,0.916,333,2004-02-01,NaN,Male,Other,CSP50,Professional,46.0,0.0,50.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0
2,0.550,173,2004-05-15,2004-12-03,Male,Other,CSP50,Private+trip to office,32.0,0.0,68.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0
3,0.089,364,2004-11-29,NaN,Female,Other,CSP55,Private+trip to office,52.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0
4,0.233,426,2004-02-07,2004-05-01,Male,Other,CSP60,Private,57.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0


В предыдущем уроке мы заметили отрицательную величину убытка для некоторых наблюдений. Заметим, что для всех таких полисов переменная "ClaimInd" принимает только значение 0. Поэтому заменим все соответствующие значения "ClaimAmount" нулями.

In [8]:
NegClaimAmount = df.loc[df.ClaimAmount < 0, ['ClaimAmount','ClaimInd']]
print('Unique values of ClaimInd:', NegClaimAmount.ClaimInd.unique())
NegClaimAmount.head()

Unique values of ClaimInd: [0.]


,ClaimAmount,ClaimInd
82,-74.206042,0.0
175,-1222.585196,0.0
177,-316.288822,0.0
363,-666.758610,0.0
375,-1201.600604,0.0


In [0]:
df.loc[df.ClaimAmount < 0, 'ClaimAmount'] = 0

Перекодируем переменные типа `object` с помощью числовых значений

In [0]:
def SeriesFactorizer(series):
    series, unique = pd.factorize(series)
    reference = {x: i for x, i in enumerate(unique)}
    print(reference)
    return series, reference

In [11]:
df.Gender, GenderRef = SeriesFactorizer(df.Gender)

{0: 'Male', 1: 'Female'}


In [12]:
df.MariStat, MariStatRef = SeriesFactorizer(df.MariStat)

{0: 'Other', 1: 'Alone'}


Для переменных, содержащих более 2 значений, различия между которыми не могут упорядочены, используем фиктивные переменные (one-hot encoding).

**NB**: В H2O не рекомендуется использовать one-hot encoding, поскольку данный фреймворк корректно работает с категориальными признаками, тогда как применение one-hot encoding приводит к неэффективности. Тем не менее, используем здесь фиктивные переменные, чтобы в дальнейшем сохранить возможность сравнения результатов построенных моделей.

In [13]:
list(df.VehUsage.unique())

['Professional', 'Private+trip to office', 'Private', 'Professional run']

In [14]:
VU_dummies = pd.get_dummies(df.VehUsage, prefix='VehUsg', drop_first=False)
VU_dummies.head()

,VehUsg_Private,VehUsg_Private+trip to office,VehUsg_Professional,VehUsg_Professional run
0,0,0,1,0
1,0,0,1,0
2,0,1,0,0
3,0,1,0,0
4,1,0,0,0


Фактор "SocioCateg" содержит информацию о социальной категории в виде кодов классификации CSP. Агрегируем имеющиеся коды до 1 знака, а затем закодируем их с помощью one-hot encoding.

[Wiki](https://fr.wikipedia.org/wiki/Professions_et_cat%C3%A9gories_socioprofessionnelles_en_France#Cr%C3%A9ation_de_la_nomenclature_des_PCS)

[Более подробный классификатор](https://www.ast74.fr/upload/administratif/liste-des-codes-csp-copie.pdf)

In [15]:
df['SocioCateg'].unique()

array(['CSP50', 'CSP55', 'CSP60', 'CSP48', 'CSP6', 'CSP66', 'CSP1',
       'CSP46', 'CSP21', 'CSP47', 'CSP42', 'CSP37', 'CSP22', 'CSP3',
       'CSP49', 'CSP20', 'CSP2', 'CSP40', 'CSP7', 'CSP26', 'CSP65',
       'CSP41', 'CSP17', 'CSP57', 'CSP56', 'CSP38', 'CSP51', 'CSP59',
       'CSP30', 'CSP44', 'CSP61', 'CSP63', 'CSP45', 'CSP16', 'CSP43'],
      dtype=object)

In [0]:
df['SocioCateg'] = df.SocioCateg.str.slice(0,4)

In [17]:
pd.DataFrame(df.SocioCateg.value_counts().sort_values()).rename({'SocioCateg': 'Frequency'}, axis=1)

,Frequency
CSP7,3
CSP3,313
CSP2,798
CSP1,1065
CSP4,2499
CSP6,6875
CSP5,20224


In [0]:
df = pd.get_dummies(df, columns=['VehUsage','SocioCateg'])

Теперь, когда большинство переменных типа `object` обработаны, исключим их из набора данных за ненадобностью.

In [0]:
df = df.select_dtypes(exclude=['object'])

Также создадим такую переменную, как квадрат возраста.

In [20]:
df['DrivAgeSq'] = df.DrivAge.apply(lambda x: x**2)
df.head()

,Exposure,LicAge,Gender,MariStat,DrivAge,HasKmLimit,BonusMalus,ClaimAmount,ClaimInd,ClaimNbResp,ClaimNbNonResp,ClaimNbParking,ClaimNbFireTheft,ClaimNbWindscreen,OutUseNb,RiskArea,VehUsage_Private,VehUsage_Private+trip to office,VehUsage_Professional,VehUsage_Professional run,SocioCateg_CSP1,SocioCateg_CSP2,SocioCateg_CSP3,SocioCateg_CSP4,SocioCateg_CSP5,SocioCateg_CSP6,SocioCateg_CSP7,DrivAgeSq
0,0.083,332,0,0,46.0,0.0,50.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,0,0,1,0,0,0,0,0,1,0,0,2116.0
1,0.916,333,0,0,46.0,0.0,50.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,0,0,1,0,0,0,0,0,1,0,0,2116.0
2,0.550,173,0,0,32.0,0.0,68.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,7.0,0,1,0,0,0,0,0,0,1,0,0,1024.0
3,0.089,364,1,0,52.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0,1,0,0,0,0,0,0,1,0,0,2704.0
4,0.233,426,0,0,57.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,1,0,0,0,0,0,0,0,0,1,0,3249.0


Для моделирования частоты убытков сгенерируем показатель как сумму индикатора того, что убыток произошел ("ClaimInd") и количества заявленных убытков по различным видам ущерба за 4 предшествующих года ("ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen").

В случаях, если соответствующая величина убытка равняется нулю, сгенерированную частоту также обнулим.

In [0]:
df['ClaimsCount'] = df.ClaimInd + df.ClaimNbResp + df.ClaimNbNonResp + df.ClaimNbParking + df.ClaimNbFireTheft + df.ClaimNbWindscreen
df.loc[df.ClaimAmount == 0, 'ClaimsCount'] = 0
df.drop(["ClaimNbResp", "ClaimNbNonResp", "ClaimNbParking", "ClaimNbFireTheft", "ClaimNbWindscreen"], axis=1, inplace=True)

In [22]:
pd.DataFrame(df.ClaimsCount.value_counts()).rename({'ClaimsCount': 'Policies'}, axis=1)

,Policies
0.0,28656
2.0,1017
1.0,875
3.0,681
4.0,359
5.0,136
6.0,42
7.0,9
8.0,2


In [23]:
import plotly.express as px
fig = px.scatter(df, x='ClaimsCount', y='ClaimAmount', title='Зависимость между частотой и величиной убытков')
fig.show()

Для моделирования среднего убытка можем рассчитать его как отношение величины убытков к их частоте.

In [0]:
dfAC = df[df.ClaimsCount > 0].copy()
dfAC['AvgClaim'] = dfAC.ClaimAmount/dfAC.ClaimsCount

## Разделение набора данных на обучающую, валидационную и тестовую выборки

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
# Разбиение датасета для частоты на train/val/test

x_train_c, x_test_c, y_train_c, y_test_c = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount'], axis=1), df.ClaimsCount, test_size=0.3, random_state=1)
x_valid_c, x_test_c, y_valid_c, y_test_c = train_test_split(x_test_c, y_test_c, test_size=0.5, random_state=1)

In [0]:
# Разбиение датасета для среднего убытка на train/val/test 

x_train_ac, x_test_ac, y_train_ac, y_test_ac = train_test_split(dfAC.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount', 'AvgClaim'], axis=1), dfAC.AvgClaim, test_size=0.3, random_state=1)
x_valid_ac, x_test_ac, y_valid_ac, y_test_ac = train_test_split(x_test_ac, y_test_ac, test_size=0.5, random_state=1)

## Обобщенные линейные модели (Generalized Linear Models, GLM)

### Теория

Пусть $y$ – целевая переменная, $X$ – матрица объясняющих переменных, $\beta$ – вектор параметров модели.

Матрица $X$ составлена из всех векторов наблюдений $x_i$, каждый из которых представляет собой объясняющую переменную.

Основные компоненты обобщенной линейной модели:
* Систематическая компонента $\eta$:
    * $\eta = X\beta \hspace{10pt}(=\beta_0+\beta_1x_1+\beta_2x_2+\dots+\beta_nx_n)$.
* Случайная компонента $y$:
    * Элементы вектора $y$ – независимые одинаково распределенные случайные величины, имеющие функцию плотности распределения $f(y;\theta,\phi)$ из экспоненциального семейства.
    * Распределения из экспоненциального семейства имеют параметры $\theta$ (характеристика среднего) и $\phi$ (характеристика дисперсии). В общем виде данные распределения могут быть определены:
    $$f_i(y_i;\theta_i,\phi)=\exp\left\lbrace \frac{y_i\theta_i-b(\theta_i)}{a_i(\phi)} + c(y_i, \phi) \right\rbrace,$$
    где $a_i(\phi)$, $b(\theta_i)$ и $c(y_i, \phi)$ некоторые функции.
    * Для распределений из данного семейства дисперсия является функцией от среднего.
    * Экспоненциальное семейство включает распределения нормальное, экспоненциальное, Пуассона, гамма, хи-квадрат, бета и другие.
* Функция связи $g$:
    * $\mathbb{E}\left[y\right]=\mu=g^{-1}\left(\eta\right)$, $\mu$ – математическое ожидание $y$;
    * $g$ – монотонная дифференцируемая функция.




#### GLM с распределением Пуассона

Регрессия Пуассона обычно используется в случаях, когда зависимая переменная представляет собой счетные значения и ошибки предполагаются распределенными в соответствии с распределением Пуассона. Зависимая переменная должна быть неотрицательной.

Функции связи между таргетом и объясняющими переменными предполагается логарифмической:
$$g(\eta) = \ln(\eta) \Rightarrow \hat {y} = e^{x{^T}\beta + {\beta_{0}}}.$$

Модель оценивается методом максимального правдоподобия, функция логарифма правдоподобия с учетом штрафа регуляризации эластичной сети имеет вид:
$$\max_{\beta,\beta_0} \frac{1}{N} \sum_{i=1}^{N} \Big( y_i(x_{i}^{T}\beta + \beta_0) - e^{x{^T_i}\beta + {\beta_0}} \Big)- \lambda \Big( \alpha||\beta||_1 + \dfrac {1} {2}(1 - \alpha)||\beta||^2_2 \Big),$$

где 
* $\lambda$ – параметр, отвечающий за силу регуляризации. $\lambda\in\mathbb{R}^{+}$;
* $\alpha$ – параметр, отвечающий за распределение штрафов регуляризации между нормой 1 ($\ell_1$) и нормой 2 ($\ell_2$). $\alpha\in[0,1]$;
* $||\beta||{_1}$ – штраф регуляризации $\ell_1$ (LASSO). $||\beta||{_1} = \sum{^p_{k=1}} |\beta{_k}|$;
* $||\beta||{_2}$ – штраф регуляризации $\ell_2$ (Ridge). $||\beta||{_2} = \sum{^p_{k=1}} \beta{^2_k}$.

Тогда соответствующая метрика _Deviance_ имеет вид:
$$D = -2 \sum_{i=1}^{N} \big( y_i \text{ln}(y_i / \hat {y}_i) - (y_i - \hat {y}_i) \big).$$

##### Вывод функции правдоподобия для GLM с распределением Пуассона (без регуляризации)

**NB:** Ниже $\lambda$ не имеет отношения к вышеупомянутому одноименному параметру регуляризации.

Напомним, что функция вероятности для распределения Пуассона имеет вид:
$$p(k;\lambda) = \frac{\lambda^k e^{-\lambda}}{k!}, \hspace{10pt} \lambda\in\mathbb{R}^{+}.$$
Также, для распределения Пуассона справедливо, что:
$$\mathbb{E}\left[k\right] = Var(k) = \lambda.$$
Тогда, для оценивания коэффициентов нашей модели необходимо максимизировать правдоподобие (совместную условную вероятность при имеющихся данных), что данные имеют распределение Пуассона:
$$p(y_1,\dots,y_n|x_1,\dots,x_n;\beta_0,\beta) = \prod_{i=1}^{N}\frac{e^{y_i(x_i{^T}\beta + {\beta_{0}})} e^{-e^{x_i{^T}\beta + {\beta_{0}}}}}{y_i!} = L(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n).$$
Для упрощения задачи оптимизации перейдем к логарифму правдоподобия:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(y_i(x_i{^T}\beta + {\beta_{0}}) -e^{x_i{^T}\beta + {\beta_{0}}}-\ln(y_i!)\right).$$
Поскольку величина $\ln(y_i!)$ не зависит от выбора параметров, можно упростить задачу:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(y_i(x_i{^T}\beta + {\beta_{0}}) -e^{x_i{^T}\beta + {\beta_{0}}}\right).$$
Далее численно решается задача оптимизации для определения параметров модели:
$$\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta_0} = 0,\\\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta} = 0.$$
Обычно минимизируется отрицательное правдоподобие, которое является выпуклой функцией.

##### Вывод метрики Deviance для GLM с распределением Пуассона

Метрика Deviance представляет собой отношение правдоподобия между двумя моделями: рассматриваемой моделью и "идеальной" моделью, в которая бы идеально предсказывала бы зависимую переменную.
$$Deviance = 2(\ell_{ideal} - \ell_{model})$$

В качестве такой "идеальной модели" может использоваться сама зависимая переменная. Тогда, логарифм правдоподобия "идеальной модели" для GLM с распределением Пуассона имеет вид:
$$\ell_{ideal} = \sum_{i=1}^{N}\left(y_i \ln(y_i) -y_i-\ln(y_i!)\right).$$

Из приведенного выше вывода правдоподобия для рассматриваемой модели, мы можем записать, обозначив $\hat{y}_i = e^{x{^T}\beta + {\beta_{0}}}$:
$$\ell_{model} = \sum_{i=1}^{N}\left(y_i \ln(\hat{y}_i) -\hat{y}_i-\ln(y_i!)\right).$$

Тогда получаем,
$$Deviance = 2\sum_{i=1}^{N}\left(y_i \ln(y_i) -y_i - y_i \ln(\hat{y}_i) +\hat{y}_i\right) = -2\sum_{i=1}^{N}\left(y_i \ln(y_i/\hat{y}_i) - (y_i -\hat{y}_i)\right).$$


#### GLM с гамма-распределением

GLM с гамма-распределением используется для моделирования положительной непрерывной зависимой переменной, когда ее условная дисперсия увеличивается вместе со средним значением, но коэффициент вариации зависимой переменной предполагается постоянным.

Обычно GLM с гамма-распределением используются с логарифмической или обратной функциями связи:
$$g(\eta) = \ln(\eta);\hspace{20pt}g(\eta) = \frac{1}{\eta}.$$

Модель оценивается методом максимального правдоподобия, функция логарифма правдоподобия (для обратной функции связи) с учетом штрафа регуляризации эластичной сети имеет вид:
$$\max_{\beta,\beta_0} - \frac{1}{N} \sum_{i=1}^{N} \frac{y_i}{x{^T_i}\beta + \beta_0} + \text{ln} \big( x{^T_i}\beta + \beta_0 \big ) - \lambda \Big( \alpha||\beta||_1 + \dfrac {1} {2}(1 - \alpha)||\beta||^2_2 \Big),$$

где 
* $\lambda$ – параметр, отвечающий за силу регуляризации. $\lambda\in\mathbb{R}^{+}$;
* $\alpha$ – параметр, отвечающий за распределение штрафов регуляризации между нормой 1 ($\ell_1$) и нормой 2 ($\ell_2$). $\alpha\in[0,1]$;
* $||\beta||{_1}$ – штраф регуляризации $\ell_1$ (LASSO). $||\beta||{_1} = \sum{^p_{k=1}} |\beta{_k}|$;
* $||\beta||{_2}$ – штраф регуляризации $\ell_2$ (Ridge). $||\beta||{_2} = \sum{^p_{k=1}} \beta{^2_k}$.

Соответствующая метрика _Deviance_ имеет вид:
$$D = 2 \sum_{i=1}^{N} - \text{ln} \bigg (\dfrac {y_i} {\hat {y}_i} \bigg) + \dfrac {(y_i - \hat{y}_i)} {\hat {y}_i}.$$




##### Вывод функции правдоподобия для GLM с Гамма-распределением и логарифмической функцией связи (без регуляризации)

**NB:** Ниже $\alpha$ и $\lambda$ не имеют отношения к вышеупомянутым одноименным параметрам регуляризации.

Напомним, что функция плотности вероятности для Гамма-распределения имеет вид:
$$f(x;\alpha,\lambda) = 
\begin{cases}
\frac{\alpha^\lambda}{\Gamma(\lambda)}x^{\lambda-1}e^{-\alpha x},&x \ge 0\\
0,& x <0
\end{cases},\hspace{20pt} \Gamma(x) = \int_0^{\infty}x^{\lambda-1}e^{-x}dx.
$$
Также, для Гамма-распределения справедливо, что:
$$ \mathbb{E}[x] = \frac{\lambda}{\alpha},\hspace{15pt} Var(x) = \frac{\lambda}{\alpha^2}.$$

Для оценивания GLM удобно параметризовать данное распределение иначе:
$$ \mathbb{E}[x] = \frac{\lambda}{\alpha} = \mu,\hspace{15pt} Var(x) = \frac{\lambda}{\alpha^2} = \frac{\mu^2}{\lambda}.$$
Тогда,
$$f(x;\mu,\lambda) = \frac{1}{\Gamma(\lambda)}\alpha^\lambda x^{\lambda-1}e^{-\alpha x} = \frac{1}{\Gamma(\lambda)}\left(\frac{\lambda}{\mu}\right)^\lambda x^{\lambda-1}e^{-\left(\frac{\lambda}{\mu}\right)x} = \frac{1}{x\cdot\Gamma(\lambda)}\left(\frac{\lambda x}{\mu}\right)^\lambda e^{-\frac{\lambda x}{\mu}},\hspace{15pt} x\ge 0.$$

Также напомним, что для GLM с функцией связи $\ln(x)$ предсказание (оценка математического ожидания) имеет вид $\hat {y} = e^{x{^T}\beta + {\beta_{0}}}$.


Тогда, для оценивания коэффициентов нашей модели необходимо максимизировать правдоподобие (совместную условную вероятность при имеющихся данных), что данные имеют Гамма-распределение, в предположении известного параметра $\lambda$:
$$p(y_1,\dots,y_n|x_1,\dots,x_n;\beta_0,\beta) = \prod_{i=1}^{N}\frac{1}{y_i\cdot\Gamma(\lambda)}\left(\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)^\lambda e^{-\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}} = L(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n).$$
Для упрощения задачи оптимизации перейдем к логарифму правдоподобия:
\begin{align}
\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) &= \sum_{i=1}^{N}\left(-\ln(\Gamma(\lambda))-\ln(y_i)+ \lambda\ln\left(\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)\\ &= \sum_{i=1}^{N}\left(-\ln(\Gamma(\lambda))-\ln(y_i) + \lambda\ln\left(\lambda\right)+\lambda\ln\left(y_i\right)-\lambda\left({x_i{^T}\beta + {\beta_{0}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right)\\ &= \sum_{i=1}^{N}\left((\lambda-1)\ln\left(y_i\right)-\ln(\Gamma(\lambda)) + \lambda\ln\left(\lambda\right)-\lambda\left({x_i{^T}\beta + {\beta_{0}}}\right) -\frac{\lambda y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right).
\end{align}
Тогда обозначим слагаемые, не зависящие от параметров модели ($\beta_0$, $\beta$), как некоторую константу $C$, получаем:
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = \sum_{i=1}^{N}\left(-\lambda\left(\left({x_i{^T}\beta + {\beta_{0}}}\right) +\frac{y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right) + C(y_i, \lambda)\right),$$
Таким образом, требуется максимизировать
$$\ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n) = -\sum_{i=1}^{N}\left(\left({x_i{^T}\beta + {\beta_{0}}}\right) + \frac{y_i}{e^{x_i{^T}\beta + {\beta_{0}}}}\right),$$
Далее численно решается задача оптимизации для определения параметров модели:
$$\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta_0} = 0,\\\frac{\partial \ell(\beta_0,\beta|y_1,\dots,y_n,x_1,\dots,x_n)}{\partial \beta} = 0.$$
Обычно минимизируется отрицательное правдоподобие, которое является выпуклой функцией.

##### Вывод метрики Deviance для GLM с Гамма-распределением

Метрика Deviance представляет собой отношение правдоподобия между двумя моделями: рассматриваемой моделью и "идеальной" моделью, в которая бы идеально предсказывала бы зависимую переменную.
$$Deviance = 2(\ell_{ideal} - \ell_{model})$$

В качестве такой "идеальной модели" может использоваться сама зависимая переменная. Тогда, логарифм правдоподобия "идеальной модели" для GLM с распределением Пуассона имеет вид:
$$\ell_{ideal} = -\sum_{i=1}^{N}\left(1 + \ln(y_i)\right).$$

Из приведенного выше вывода правдоподобия для рассматриваемой модели, мы можем записать, обозначив $\hat{y}_i = e^{x{^T}\beta + {\beta_{0}}}$:
$$\ell_{model} = -\sum_{i=1}^{N}\left(\frac{y_i}{\hat{y}_i} + \ln(\hat{y}_i)\right).$$

Тогда получаем,
$$Deviance = 2\sum_{i=1}^{N}\left(- 1 - \ln(y_i) + \frac{y_i}{\hat{y}_i} + \ln(\hat{y}_i)\right) = 2\sum_{i=1}^{N}\left(-\ln\left(\frac{y_i}{\hat{y}_i}\right)+\frac{y_i-\hat{y}_i}{\hat{y}_i} \right).$$

#### Дополнительная литература по GLM

*   [P. McCullagh, John A. Nelder _"Generalized Linear Models"_](http://www.utstat.toronto.edu/~brunner/oldclass/2201s11/readings/glmbook.pdf)
*   [D. Anderson, et al. _"A practitioner’s guide to generalized linear models"_](https://www.casact.org/pubs/dpp/dpp04/04dpp1.pdf)
*   [E. Ohlsson, B. Johansson _"Non-life insurance pricing with generalized linear models"_](https://www.springer.com/gp/book/9783642107900)
*   [P. De Jong, G. Heller _"Generalized linear models for insurance data"_](https://feb.kuleuven.be/public/u0017833/boek.pdf)




### Установка H2O на Google Colaboratory и инициализация

In [28]:
!apt-get install default-jre

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


In [29]:
!java -version

openjdk version "11.0.6" 2020-01-14
OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)


In [30]:
!pip install h2o

     |████████████████████████████████| 129.6MB 77kB/s 
  Created wheel for h2o: filename=h2o-3.30.0.2-py2.py3-none-any.whl size=129672980 sha256=7d66549c20adf418220c760e753a58185f7431800b12007bd535a62d26eae0f2
  Stored in directory: /root/.cache/pip/wheels/35/f0/d4/2090fb81ef10fd3dedffd1386a8a4d79b882e8e978fbee4e14
Successfully built h2o


In [31]:
import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpznapoaix
  JVM stdout: /tmp/tmpznapoaix/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpznapoaix/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,1 day
H2O_cluster_name:,H2O_from_python_unknownUser_8k030v
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


### Построение GLM для частоты страховых случаев

In [32]:
# Преобразование в H2O-Frame

h2o_train_c = h2o.H2OFrame(pd.concat([x_train_c, y_train_c], axis=1))
h2o_valid_c = h2o.H2OFrame(pd.concat([x_valid_c, y_valid_c], axis=1))
h2o_test_c = h2o.H2OFrame(pd.concat([x_test_c, y_test_c], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [120]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_poisson = H2OGeneralizedLinearEstimator(family = "poisson", link = "Log", nfolds=5)
glm_poisson.train(y="ClaimsCount", x = h2o_train_c.names[1:-1], training_frame = h2o_train_c, validation_frame = h2o_valid_c, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [121]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_poisson.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,poisson,log,"Elastic Net (alpha = 0.5, lambda = 1.431E-4 )",19,19,3,Key_Frame__upload_8369c4ea9e1dafb84536af1d2f714218.hex


In [35]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_poisson.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.53740376,0.0047916896,0.53524923,0.5412414,0.5372962,0.54258335,0.53064865
1,mean_residual_deviance,1.3158892,0.021155676,1.3055145,1.3210299,1.3169911,1.3466471,1.2892636
2,mse,0.8279212,0.032375634,0.8171252,0.8531306,0.83747405,0.8550676,0.7768088
3,null_deviance,2659.7305,54.800076,2633.016,2651.9143,2682.3674,2738.6646,2592.69
4,r2,0.009658365,0.008435081,0.010976392,0.013328364,0.020249523,0.006103629,-0.0023660874
5,residual_deviance,2607.433,51.158833,2583.2227,2583.4233,2596.2742,2698.0278,2576.2183
6,rmse,0.90976053,0.017922785,0.9039498,0.9236507,0.9151361,0.92469865,0.88136756
7,rmsle,0.42632693,0.0028254073,0.42435154,0.42720968,0.42562506,0.43077013,0.42367822


In [36]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_poisson._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-2.596659,-1.209577
1,LicAge,0.000790,0.125331
2,Gender,0.044403,0.021295
3,MariStat,-0.016725,-0.006009
4,DrivAge,0.014460,0.210630
5,HasKmLimit,-0.316167,-0.078299
6,BonusMalus,0.014597,0.225351
7,OutUseNb,0.062204,0.040201
8,RiskArea,0.008566,0.018748
9,VehUsage_Private,-0.090148,-0.041374


In [37]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_poisson.coef_norm()
for x in range(len(glm_poisson.cross_validation_models())):
    pmodels[x] = glm_poisson.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-1.20958,-1.21020,-1.20822,-1.20675,-1.22247,-1.20529
LicAge,0.12533,0.15766,0.14445,0.12721,0.16369,0.04504
Gender,0.02130,0.01426,0.01148,0.01475,0.03834,0.02820
MariStat,-0.00601,0.00321,-0.00791,-0.00671,-0.01170,-0.01001
DrivAge,0.21063,0.26897,0.16650,0.16904,0.06756,0.28036
HasKmLimit,-0.07830,-0.09543,-0.05690,-0.08916,-0.10525,-0.04890
BonusMalus,0.22535,0.22504,0.22767,0.21125,0.22354,0.23574
OutUseNb,0.04020,0.02487,0.02922,0.04258,0.05435,0.04909
RiskArea,0.01875,0.04435,0.01737,0.02296,0.00561,0.00383
VehUsage_Private,-0.04137,-0.03790,-0.04867,-0.02326,-0.05125,-0.03951


In [38]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

c_train_pred = glm_poisson.predict(h2o_train_c).as_data_frame()
c_valid_pred = glm_poisson.predict(h2o_valid_c).as_data_frame()
c_test_pred = glm_poisson.predict(h2o_test_c).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [0]:
# Сохранение обученной модели

model_glm_poisson = h2o.save_model(model=glm_poisson, path="/content/drive/My Drive/Colab Notebooks/", force=True)

In [41]:
model_glm_poisson

'/content/drive/My Drive/Colab Notebooks/GLM_model_python_1588242173575_1'

### Построение GLM для среднего убытка

In [42]:
# Преобразование в H2O-Frame

h2o_train_ac = h2o.H2OFrame(pd.concat([x_train_ac, y_train_ac], axis=1))
h2o_valid_ac = h2o.H2OFrame(pd.concat([x_valid_ac, y_valid_ac], axis=1))
h2o_test_ac = h2o.H2OFrame(pd.concat([x_test_ac, y_test_ac], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [43]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_gamma = H2OGeneralizedLinearEstimator(family = "gamma", link = "Log", nfolds=5)
glm_gamma.train(y="AvgClaim", x = h2o_train_ac.names[1:-1], training_frame = h2o_train_ac, validation_frame = h2o_valid_ac, weights_column = "Exposure")

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [44]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_gamma.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,gamma,log,"Elastic Net (alpha = 0.5, lambda = 2.745E-4 )",19,17,3,Key_Frame__upload_ac4a01c6ef097dbd55a5ff4ef62f49b5.hex


In [45]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_gamma.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,1199.3129,89.616104,1303.5127,1062.3893,1243.6735,1209.0334,1177.9551
1,mean_residual_deviance,2.1386175,0.161109,2.3798213,1.9647441,2.1566708,2.0202878,2.171563
2,mse,6103705.5,1618838.2,7572372.0,3647338.0,5676591.0,7555609.0,6066617.5
3,null_deviance,565.0664,61.98265,671.63776,511.56805,542.9829,540.0845,559.05865
4,r2,-0.007195876,0.020714408,-0.005907951,-0.025151098,0.010600132,0.015423417,-0.03094388
5,residual_deviance,570.7998,69.515526,682.57794,523.5532,544.07196,512.0601,591.73566
6,rmse,2451.1904,345.27393,2751.7944,1909.8005,2382.5598,2748.7468,2463.0505
7,rmsle,1.7877306,0.042351373,1.8462442,1.7929554,1.774244,1.7288489,1.7963603


In [46]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)

glm_gamma._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,7.590457,6.997392
1,LicAge,-0.000373,-0.058712
2,Gender,-0.001931,-0.000928
3,MariStat,-0.027457,-0.010107
4,DrivAge,-0.041143,-0.588286
5,HasKmLimit,0.027892,0.006046
6,BonusMalus,-0.000903,-0.015472
7,OutUseNb,-0.016520,-0.011754
8,RiskArea,0.051934,0.116652
9,VehUsage_Private,0.343891,0.152261


In [47]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_gamma.coef_norm()
for x in range(len(glm_gamma.cross_validation_models())):
    pmodels[x] = glm_gamma.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,6.99739,6.96514,7.01641,6.98456,7.00410,6.99190
LicAge,-0.05871,0.05692,-0.12399,-0.17536,0.00079,-0.03830
Gender,-0.00093,0.04451,-0.00051,-0.02457,-0.00977,-0.01268
MariStat,-0.01011,-0.06530,-0.02955,-0.01900,0.03418,0.01865
DrivAge,-0.58829,-0.53114,-0.43011,-0.60257,-0.42585,-0.96298
HasKmLimit,0.00605,-0.02806,-0.00088,0.01089,0.02937,0.01497
BonusMalus,-0.01547,0.04086,0.02623,-0.03509,-0.01705,-0.09404
OutUseNb,-0.01175,0.02015,-0.08280,0.01081,-0.02378,0.00138
RiskArea,0.11665,0.16901,0.09459,0.10417,0.10938,0.11126
VehUsage_Private,0.15226,0.13617,0.18864,0.10032,0.07749,0.22671


In [48]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ac_train_pred = glm_gamma.predict(h2o_train_ac).as_data_frame()
ac_valid_pred = glm_gamma.predict(h2o_valid_ac).as_data_frame()
ac_test_pred = glm_gamma.predict(h2o_test_ac).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [0]:
# Сохранение обученной модели

model_glm_gamma = h2o.save_model(model=glm_gamma, path="/content/drive/My Drive/Colab Notebooks/", force=True)

In [50]:
model_glm_gamma

'/content/drive/My Drive/Colab Notebooks/GLM_model_python_1588242173575_2'

### Использование GLM моделей

In [51]:
h2o_df = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [52]:
df['CountPredicted'] = glm_poisson.predict(h2o_df).as_data_frame()
df['AvgClaimPredicted'] = glm_gamma.predict(h2o_df).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [53]:
df['BurningCost'] = df.CountPredicted * df.AvgClaimPredicted
df[['CountPredicted', 'AvgClaimPredicted', 'BurningCost']].head()

,CountPredicted,AvgClaimPredicted,BurningCost
0,0.366791,1072.350140,393.328284
1,0.367081,1071.950560,393.492633
2,0.301566,1179.393055,355.665189
3,0.268842,1041.894728,280.104550
4,0.253903,959.223130,243.549306


## * Домашнее задание: GLM для прогнозирования наступления страхового случая

In [0]:
# Разбиение датасета на train/val/test

x_train_ind, x_test_ind, y_train_ind, y_test_ind = train_test_split(df.drop(['ClaimInd', 'ClaimAmount', 'ClaimsCount', 'CountPredicted', 'AvgClaimPredicted', 'BurningCost'], axis=1), df.ClaimInd, test_size=0.3, random_state=1)
x_valid_ind, x_test_ind, y_valid_ind, y_test_ind = train_test_split(x_test_ind, y_test_ind, test_size=0.5, random_state=1)

In [111]:
y_train_ind

7768     0.0
2238     0.0
29054    0.0
4799     0.0
8427     1.0
        ... 
17289    0.0
5192     0.0
12172    0.0
235      0.0
29733    0.0
Name: ClaimInd, Length: 22243, dtype: float64

In [112]:
y_train_ind.unique()

array([0., 1.])

In [113]:
# % страховых случаев в обучающей выборке
y_train_ind[y_train_ind==1].value_counts()/y_train_ind.size*100

1.0    9.751382
Name: ClaimInd, dtype: float64

видно что классы не сбалансированы

In [114]:
# Преобразование в H2O-Frame

h2o_train_ind = h2o.H2OFrame(pd.concat([x_train_ind, y_train_ind], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [115]:
h2o_train_ind['ClaimInd'].types

{'ClaimInd': 'int'}

In [0]:
# Преобразуем целевую переменную ClaimInd в категориальную при помощи метода asfactor во всех наборах данных
h2o_train_ind['ClaimInd']=h2o_train_ind['ClaimInd'].asfactor()
h2o_valid_ind['ClaimInd']=h2o_valid_ind['ClaimInd'].asfactor()
h2o_test_ind['ClaimInd']=h2o_test_ind['ClaimInd'].asfactor()

In [117]:
h2o_train_ind['ClaimInd'].types

{'ClaimInd': 'enum'}

In [0]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_poisson_ind = H2OGeneralizedLinearEstimator(family = "binomial", lambda_=0, nfolds=5)


In [146]:
glm_poisson_ind.train(y='ClaimInd', x = h2o_train_ind.names[1:-1], training_frame = h2o_train_ind, validation_frame = h2o_valid_ind)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [147]:
# Параметры модели: распределение, функция связи, гиперпараметры регуляризации, количество использованных объясняющих переменных

glm_poisson_ind.summary()


GLM Model: summary


,,family,link,regularization,number_of_predictors_total,number_of_active_predictors,number_of_iterations,training_frame
0,,binomial,logit,None,19,19,3,py_9_sid_929f


In [148]:
# Метрики качества модели - по всем данным и на кросс-валидации

glm_poisson_ind.accuracy(train=True, valid=True)

{'train': [[0.26577890501089513, 0.9024412174616734]],
 'valid': [[0.24586563836955913, 0.8988881896370883]]}

In [149]:
glm_poisson_ind.F1(train=True, valid=True)

{'train': [[0.09441205756876844, 0.19268707482993197]],
 'valid': [[0.09835945914563844, 0.2006213936972925]]}

In [163]:
glm_poisson_ind.auc(train=True, valid=True)

{'train': 0.5674088858774402, 'valid': 0.5583568510540046}

In [162]:
glm_poisson_ind.cross_validation_metrics_summary().as_data_frame()

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.5911952,0.15260948,0.5948706,0.6840845,0.7000683,0.64903957,0.32791328
1,auc,0.55709594,0.015153878,0.5507322,0.5487001,0.5807065,0.5629404,0.5424004
2,aucpr,0.11573358,0.0084303785,0.122268125,0.11901,0.12303958,0.11111061,0.103239566
3,err,0.40880474,0.15260948,0.40512937,0.31591552,0.29993168,0.35096046,0.6720867
4,err_count,1816.4,670.4393,1785.0,1451.0,1317.0,1553.0,2976.0
5,f0point5,0.14443615,0.014511391,0.15341058,0.14900948,0.15764095,0.14127381,0.12084592
6,f1,0.19415548,0.012355967,0.2070191,0.19254313,0.2051901,0.18903394,0.17699115
7,f2,0.30004036,0.023960982,0.31821907,0.2720126,0.29381266,0.28557906,0.3305785
8,lift_top_group,1.3239833,0.38851523,1.4582506,1.0829482,1.9377759,1.176236,0.9647059
9,logloss,0.31818274,0.013830046,0.3395706,0.3246221,0.3082373,0.31116313,0.30732054


In [141]:
# Таблица коэффициентов модели (в зависимости от модели могут выводиться также стандартная ошибка, z-score и p-value)
glm_poisson_ind._model_json['output']['coefficients_table'].as_data_frame()

,names,coefficients,standardized_coefficients
0,Intercept,-3.054256,-2.245676
1,LicAge,-0.000098,-0.015538
2,Gender,0.034735,0.016666
3,MariStat,0.039062,0.014040
4,DrivAge,0.015620,0.228236
5,HasKmLimit,-0.282242,-0.071671
6,BonusMalus,0.007835,0.122450
7,OutUseNb,0.060339,0.040238
8,RiskArea,0.012581,0.027466
9,VehUsage_Private,-0.087640,-0.040280


In [150]:
glm_poisson_ind.coef()

{'BonusMalus': 0.007835396501136151,
 'DrivAge': 0.015620248281283609,
 'DrivAgeSq': -0.00017839450608473693,
 'Gender': 0.03473477967397766,
 'HasKmLimit': -0.282241988776335,
 'Intercept': -3.054256390482206,
 'LicAge': -9.769444170359235e-05,
 'MariStat': 0.03906164836967898,
 'OutUseNb': 0.06033868176203341,
 'RiskArea': 0.012580730185240264,
 'SocioCateg_CSP1': -0.052259377145140534,
 'SocioCateg_CSP2': -0.196890215165023,
 'SocioCateg_CSP3': -0.11108464709571444,
 'SocioCateg_CSP4': 0.059741394358008716,
 'SocioCateg_CSP5': -0.042868077453923535,
 'SocioCateg_CSP6': 0.07630962762400227,
 'VehUsage_Private': -0.08764030155226162,
 'VehUsage_Private+trip to office': -0.07406705050232602,
 'VehUsage_Professional': 0.20355772179472878,
 'VehUsage_Professional run': 0.32516646193773246}

In [152]:
# Таблица нормированных коэффициентов по всем данным и на кросс-валидации

pmodels = {}
pmodels['overall'] = glm_poisson_ind.coef_norm()
for x in range(len(glm_poisson_ind.cross_validation_models())):
    pmodels[x] = glm_poisson_ind.cross_validation_models()[x].coef_norm()
pd.DataFrame.from_dict(pmodels).round(5)

,overall,0,1,2,3,4
Intercept,-2.24568,-2.27482,-2.25492,-2.23252,-2.23749,-2.23380
LicAge,-0.01554,-0.00191,-0.07071,-0.03494,0.01409,0.01695
Gender,0.01667,0.03803,-0.00257,0.01036,0.02304,0.01406
MariStat,0.01404,0.00700,-0.00601,0.01549,0.02985,0.02374
DrivAge,0.22824,0.11148,0.24187,0.40846,0.10725,0.27459
HasKmLimit,-0.07167,-0.09252,-0.07104,-0.05537,-0.05692,-0.08550
BonusMalus,0.12245,0.12261,0.12724,0.11960,0.12627,0.11683
OutUseNb,0.04024,0.04546,0.04385,0.02477,0.04548,0.04086
RiskArea,0.02747,0.02724,0.03548,0.03909,0.01356,0.02206
VehUsage_Private,-0.04028,-0.06496,-0.02813,-1.67593,-0.04054,-0.04387


In [155]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ind_train_pred = glm_poisson_ind.predict(h2o_train_ind).as_data_frame()
ind_valid_pred = glm_poisson_ind.predict(h2o_valid_ind).as_data_frame()
ind_test_pred = glm_poisson_ind.predict(h2o_test_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [158]:
ind_train_pred.head() 

,predict,p0,p1
0,1,0.884099,0.115901
1,1,0.885516,0.114484
2,0,0.913044,0.086956
3,1,0.896298,0.103702
4,0,0.911373,0.088627


In [0]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

In [165]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

print('accuracy_score', '\n', 'train', accuracy_score(y_train_ind,ind_train_pred['predict']),'\n',
      'valid', accuracy_score(y_valid_ind,ind_valid_pred['predict']),'\n',
      'test', accuracy_score(y_test_ind,ind_test_pred['predict']),'\n')
      

accuracy_score 
 train 0.6168232702423234 
 valid 0.6232431298510593 
 test 0.6085588420390182 



In [166]:
print('f1_score', '\n', 'train', f1_score(y_train_ind,ind_train_pred['predict']),'\n',
      'valid', f1_score(y_valid_ind,ind_valid_pred['predict']),'\n',
      'test', f1_score(y_test_ind,ind_test_pred['predict']),'\n')

f1_score 
 train 0.18990590248075276 
 valid 0.19964349376114082 
 test 0.19707401032702238 



In [168]:
confusion_matrix(y_train_ind,ind_train_pred['predict'])


array([[12721,  7353],
       [ 1170,   999]])

In [169]:
confusion_matrix(y_valid_ind,ind_valid_pred['predict'])


array([[2747, 1539],
       [ 257,  224]])

In [170]:
confusion_matrix(y_test_ind,ind_test_pred['predict'])

array([[2672, 1624],
       [ 242,  229]])

Какие проблемы вы здесь видите? Как можно улучшить данный результат?

In [179]:
# сбалансируем классы в тренировочной выборке

from imblearn.over_sampling import SMOTE

sm=SMOTE(random_state=42, sampling_strategy=0.4,n_jobs=1)
sm


SMOTE(k_neighbors=5, kind='deprecated', m_neighbors='deprecated', n_jobs=1,
      out_step='deprecated', random_state=42, ratio=None, sampling_strategy=0.4,
      svm_estimator='deprecated')

In [180]:
x_train_balanced, y_train_balanced = sm.fit_sample(x_train_ind, y_train_ind)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning:

Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.



In [184]:
x_train_ind.columns

Index(['Exposure', 'LicAge', 'Gender', 'MariStat', 'DrivAge', 'HasKmLimit',
       'BonusMalus', 'OutUseNb', 'RiskArea', 'VehUsage_Private',
       'VehUsage_Private+trip to office', 'VehUsage_Professional',
       'VehUsage_Professional run', 'SocioCateg_CSP1', 'SocioCateg_CSP2',
       'SocioCateg_CSP3', 'SocioCateg_CSP4', 'SocioCateg_CSP5',
       'SocioCateg_CSP6', 'SocioCateg_CSP7', 'DrivAgeSq'],
      dtype='object')

In [189]:
y_train_ind.name

'ClaimInd'

In [0]:
x_train_ind_b=pd.DataFrame(x_train_balanced, columns=x_train_ind.columns)
y_train_ind_b=pd.Series(y_train_balanced, name=y_train_ind.name)



In [192]:
# % страховых случаев в обучающей выборке
len(y_train_balanced[y_train_balanced==1])/len(y_train_balanced)*100

28.56990356901398

In [193]:
h2o_train_ind_b = h2o.H2OFrame(pd.concat([x_train_ind_b, y_train_ind_b], axis=1))
h2o_valid_ind = h2o.H2OFrame(pd.concat([x_valid_ind, y_valid_ind], axis=1))
h2o_test_ind = h2o.H2OFrame(pd.concat([x_test_ind, y_test_ind], axis=1))

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
# Инициализируем и обучим GLM модель c кросс-валидацией

glm_poisson_ind_b = H2OGeneralizedLinearEstimator(family = "binomial", lambda_=0, nfolds=5)

In [195]:
glm_poisson_ind_b.train(y='ClaimInd', x = h2o_train_ind_b.names[1:-1], training_frame = h2o_train_ind_b, validation_frame = h2o_valid_ind)

glm Model Build progress: |███████████████████████████████████████████████| 100%


In [196]:
# Построение прогнозных значений для обучающей, валидационной и тестовой выборок

ind_train_pred = glm_poisson_ind_b.predict(h2o_train_ind_b).as_data_frame()
ind_valid_pred = glm_poisson_ind_b.predict(h2o_valid_ind).as_data_frame()
ind_test_pred = glm_poisson_ind_b.predict(h2o_test_ind).as_data_frame()

glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%


In [197]:
# Выведем импортированные выше метрики классификации для обучающей, валидационной и тестовой выборок

print('accuracy_score', '\n', 'train', accuracy_score(y_train_ind_b,ind_train_pred['predict']),'\n',
      'valid', accuracy_score(y_valid_ind,ind_valid_pred['predict']),'\n',
      'test', accuracy_score(y_test_ind,ind_test_pred['predict']),'\n')

accuracy_score 
 train 0.5907910187524463 
 valid 0.6310048248374239 
 test 0.6123348017621145 



на валиде и тесте accuracy немного подросла. Старые значения до балансировки:

 train 0.6168232702423234 

 valid 0.6232431298510593 
 
 test 0.6085588420390182 

In [199]:
print('f1_score', '\n', 'train', f1_score(y_train_ind_b,ind_train_pred['predict']),'\n',
      'valid', f1_score(y_valid_ind,ind_valid_pred['predict']),'\n',
      'test', f1_score(y_test_ind,ind_test_pred['predict']),'\n')

f1_score 
 train 0.3940990516332983 
 valid 0.1986332574031891 
 test 0.1930131004366812 



In [200]:
from sklearn.metrics import roc_auc_score
print('roc_auc', '\n', 'train', roc_auc_score(y_train_ind_b,ind_train_pred['predict']),'\n',
      'valid', roc_auc_score(y_valid_ind,ind_valid_pred['predict']),'\n',
      'test', roc_auc_score(y_test_ind,ind_test_pred['predict']),'\n')

roc_auc 
 train 0.5532952754097422 
 valid 0.5520895280577968 
 test 0.5486202540654023 

